In [1]:
import pandas as pd
import json
import numpy as np
from gensim.models import FastText

In [3]:
# FastText 모델 학습
def train_fasttext_model(combined_data):
    # FastText 모델 학습
    model = FastText(sentences=[sentence.split() for sentence in combined_data],  # 데이터를 단어별로 분리하여 학습
                     vector_size=150,  # 단어 벡터의 차원 크기
                     window=7,         # 주변 단어의 범위
                     sg=1,             # 1은 skip-gram 방식, 0은 CBOW 방식
                     negative=3,       # negative sampling 수
                     min_count=5,      # 최소 등장 횟수
                     epochs=10)        # 학습 반복 횟수
    model.save('fasttext.model')  # 모델 저장
    return model

# 벡터화 함수 정의
def vectorize_text(text, model):
    # 텍스트를 토큰화하고 각 단어의 벡터 평균을 구함
    tokens = text.split()
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)  # 각 단어 벡터의 평균 반환
    else:
        return np.zeros(model.vector_size)  # 빈 텍스트의 경우 0 벡터 반환

# 데이터 로드 및 벡터화 후 덮어쓰기
def embedding(filename):
    # CSV 파일 로드
    df = pd.read_csv(filename)

    # 뉴스 제목과 내용 결합하여 학습용 데이터 준비
    combined_data = df['newsTitle'].fillna('') + " " + df['newsContent'].fillna('')

    # FastText 모델 학습
    model = train_fasttext_model(combined_data)

    # 벡터화 및 원래 데이터 덮어쓰기
    for index, row in df.iterrows():
        if 'newsTitle' in row and pd.notna(row['newsTitle']):
            df.at[index, 'newsTitle'] = vectorize_text(row['newsTitle'], model).tolist()
        if 'newsContent' in row and pd.notna(row['newsContent']):
            df.at[index, 'newsContent'] = vectorize_text(row['newsContent'], model).tolist()

    # 새로운 파일 경로로 저장
    new_filename = "embedded_" + filename
    df.to_csv(new_filename, index=False, encoding='utf-8')

In [4]:
# 벡터화 후 CSV 저장
embedding('newsdata_train.csv')
embedding('newsdata_test.csv')

In [ ]:
model = FastText.load('fasttext.model')
print(model.wv['서울'])
print(model.wv.most_similar('서울'))

[-0.25104544 -0.07970234 -0.02213505  0.17463736  0.30319142  0.14689553
  0.12875228  0.02686078 -0.49554014  0.18140385  0.21220806 -0.13649045
 -0.17695391  0.02300723  0.18467507  0.1247035   0.17377234 -0.3212884
  0.16452423 -0.42488682  0.399189   -0.07933641  0.12229703 -0.03248195
 -0.06627069  0.1810525  -0.06949529 -0.15336135 -0.01214663  0.49436295
  0.08129417  0.04610986  0.24088158  0.14088747  0.18017589  0.52900684
  0.0963407   0.0335203  -0.34491783  0.07212323  0.05752467 -0.3886918
  0.06200375  0.04256329  0.30373427  0.5870484  -0.22144698  0.49868852
 -0.3228535   0.06863975  0.10747646  0.04319504  0.03697788  0.19020906
 -0.5343235  -0.11278027  0.05165378  0.1289041  -0.14015892 -0.19388796
 -0.16653273 -0.07024307 -0.33946943  0.0016133   0.2699325  -0.42199978
 -0.01871256  0.55657935 -0.3300532  -0.05064172 -0.39031273 -0.08447891
 -0.00993035  0.0453406   0.25731307  0.19328137 -0.1725997  -0.18256144
 -0.41430107  0.13684863  0.16288933 -0.29569685  0.3